In [1]:
import json
import numpy as np
from sklearn.model_selection import train_test_split
import tensorflow.keras as keras

DATA_PATH = "data7c.json"

def load_data(data_path):
    
    with open(data_path, "r") as fp:
        data = json.load(fp)
        
    X = np.array(data["MFCCs"])
    y = np.array(data["label"])
    return X, y

def prepare_datasets(test_size, validation_size):
    
    # load data
    X, y = load_data(DATA_PATH)
    
    # create train/test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)

    # create train/validation split
    X_train, X_validation, y_train, y_validation = train_test_split(X_train, y_train, test_size=validation_size)

    # 3d array -> (130, 13) time bins X n_mfccs but we miss the channel (1)
    X_train = X_train[..., np.newaxis] # 4d array -> (130, 13, 1)
    X_validation = X_validation[..., np.newaxis]
    X_test = X_test[..., np.newaxis]
    
    return X_train, X_validation, X_test, y_train, y_validation, y_test


def build_model(input_shape):
    
    # create model
    model = keras.Sequential()
    
    # 1st conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape)) # kernel, grid size and activation function
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())  # speed up training
    
    # 2nd conv layer
    model.add(keras.layers.Conv2D(32, (3, 3), activation='relu')) # kernel, grid size and activation function
    model.add(keras.layers.MaxPooling2D((3, 3), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())  # speed up training
    
    # 3rd conv layer
    model.add(keras.layers.Conv2D(32, (2, 2), activation='relu')) # kernel, grid size and activation function
    model.add(keras.layers.MaxPooling2D((2, 2), strides=(2, 2), padding='same'))
    model.add(keras.layers.BatchNormalization())  # speed up training
    
    # flatten the output and feed it into dense layer
    model.add(keras.layers.Flatten())
    model.add(keras.layers.Dense(64, activation='relu'))
    model.add(keras.layers.Dropout(0.3)) # to avoid overfitting
    
    # output layer (softmax)
    model.add(keras.layers.Dense(8, activation='softmax'))
    
    return model
    

# if __name__ == "__main__"

# create train, validation and test sets
X_train, X_validation, X_test, y_train, y_validation, y_test = prepare_datasets(0.25, 0.2)

# build the CNN
input_shape = (X_train.shape[1], X_train.shape[2], X_train.shape[3])
model = build_model(input_shape)

# compile the network
optimizer = keras.optimizers.Adam(learning_rate=0.0001)
model.compile(optimizer, loss="sparse_categorical_crossentropy", metrics=['accuracy'])

# train the CNN
model.fit(X_train, y_train, validation_data=(X_validation, y_validation), batch_size=8, epochs=120)

# evaluate the CNN on the test set
test_error, test_accuracy = model.evaluate(X_test, y_test, verbose=1)
print("Accuracy on test set is: {}".format(test_accuracy))

# make a prediction on a sample
# inspired by Valerio Velardo's videos


Epoch 1/120
346/346 [==============================] - 10s 29ms/step - loss: 1.4604 - accuracy: 0.4566 - val_loss: 1.1143 - val_accuracy: 0.5882
Epoch 2/120
346/346 [==============================] - 10s 28ms/step - loss: 1.0672 - accuracy: 0.6059 - val_loss: 0.9899 - val_accuracy: 0.6084
Epoch 3/120
346/346 [==============================] - 10s 30ms/step - loss: 0.9521 - accuracy: 0.6461 - val_loss: 0.9368 - val_accuracy: 0.6460
Epoch 4/120
346/346 [==============================] - 10s 29ms/step - loss: 0.8435 - accuracy: 0.7028 - val_loss: 0.8674 - val_accuracy: 0.6922
Epoch 5/120
346/346 [==============================] - 10s 30ms/step - loss: 0.7724 - accuracy: 0.7158 - val_loss: 0.8368 - val_accuracy: 0.6792
Epoch 6/120
346/346 [==============================] - 10s 29ms/step - loss: 0.6964 - accuracy: 0.7462 - val_loss: 0.8104 - val_accuracy: 0.6850
Epoch 7/120
346/346 [==============================] - 10s 28ms/step - loss: 0.6228 - accuracy: 0.7805 - val_loss: 0.7734 - val_ac

346/346 [==============================] - 9s 27ms/step - loss: 0.0059 - accuracy: 0.9993 - val_loss: 1.3337 - val_accuracy: 0.7572
Epoch 114/120
346/346 [==============================] - 10s 28ms/step - loss: 0.0139 - accuracy: 0.9960 - val_loss: 1.3282 - val_accuracy: 0.7428
Epoch 115/120
346/346 [==============================] - 10s 29ms/step - loss: 0.0077 - accuracy: 0.9964 - val_loss: 1.3962 - val_accuracy: 0.7211
Epoch 116/120
346/346 [==============================] - 10s 29ms/step - loss: 0.0111 - accuracy: 0.9960 - val_loss: 1.3097 - val_accuracy: 0.7413
Epoch 117/120
346/346 [==============================] - 10s 29ms/step - loss: 0.0075 - accuracy: 0.9975 - val_loss: 1.2615 - val_accuracy: 0.7558
Epoch 118/120
346/346 [==============================] - 10s 28ms/step - loss: 0.0044 - accuracy: 0.9989 - val_loss: 1.3113 - val_accuracy: 0.7370
Epoch 119/120
346/346 [==============================] - 9s 27ms/step - loss: 0.0059 - accuracy: 0.9989 - val_loss: 1.4068 - val_accu